In [8]:
import pandas as pd

dataframe = pd.read_csv("Online Retail.csv", encoding = "ISO-8859-1")
dataframe["InvoiceDate"] = pd.to_datetime(dataframe["InvoiceDate"])
dataframe.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [24]:
# Calculate RFM

#Recency: How recently a customer has made a purchase
#Frequency: How often a customer makes a purchase
#Monetary Value: How much money a customer spends on purchases

dataframe['TotalSum'] = dataframe['Quantity'] * dataframe['UnitPrice']
snapshot_date = max(dataframe.InvoiceDate)
datamart = dataframe.groupby('CustomerID').agg(
    {
        'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
        'InvoiceNo': 'count',
        'TotalSum': 'sum'
    }
).round(1)

datamart.rename(

    columns={'InvoiceDate':'Recency', 'InvoiceNo': 'Frequency', 'TotalSum':'Monetary'}, 
    inplace=True
)

print(datamart)

            Recency  Frequency  Monetary
CustomerID                              
12346.0         325          2      0.00
12347.0           1        182   4310.00
12348.0          74         31   1797.24
12349.0          18         73   1757.55
12350.0         309         17    334.40
...             ...        ...       ...
18280.0         277         10    180.60
18281.0         180          7     80.82
18282.0           7         13    176.60
18283.0           3        756   2094.88
18287.0          42         70   1837.28

[4372 rows x 3 columns]


In [55]:
# Building RFM segments

r_labels = range(4, 0, -1)
r_quartiles = pd.qcut(datamart['Recency'], 4, labels=r_labels)
datamart['R'] = r_quartiles

f_labels = range(1, 5)
f_quartiles = pd.qcut(datamart['Frequency'], 4, labels=f_labels)
datamart['F'] = f_quartiles

m_labels = range(1, 5)
m_quartiles = pd.qcut(datamart['Monetary'], 4, labels=m_labels)
datamart['M'] = m_quartiles

def join_rfm(x): return str(int(x['R'])) + str(int(x['F'])) + str(int(x['M']))
datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)


datamart['RFM_Score'] = datamart[['R', 'F' ,'M']].sum(axis=1)
datamart.sort_values('RFM_Score')
datamart.head(17)

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score,General_Segment
CustomerID,,,,,,,,,
12346.0,325,2,0.000000e+00,1,1,1,111,3,Bronze
12821.0,213,6,9.272000e+01,1,1,1,111,3,Bronze
13723.0,216,6,1.998500e+02,1,1,1,111,3,Bronze
12817.0,280,4,1.660400e+02,1,1,1,111,3,Bronze
15843.0,303,8,1.189500e+02,1,1,1,111,3,Bronze
15823.0,336,17,-8.407600e+02,1,1,1,111,3,Bronze
15820.0,319,17,2.069800e+02,1,1,1,111,3,Bronze
13747.0,373,1,7.960000e+01,1,1,1,111,3,Bronze
13754.0,151,12,2.826000e+02,1,1,1,111,3,Bronze


In [53]:
# Analyzing RFM table

# 1
datamart.groupby('RFM_Segment').size().sort_values(ascending=False).head(100)

# 2
datamart[datamart['RFM_Segment'] == '111']

# 3
datamart.groupby('RFM_Score').agg(
    {
        'Recency':'mean',
        'Frequency': 'mean',
        'Monetary':['count', 'mean']
    }
).round(1)

# 4
def segment_name(df):
    if df['RFM_Score'] > 9: return 'Gold'
    if df['RFM_Score'] >= 5 and df['RFM_Score'] < 9: return 'Silver'
    if df['RFM_Score'] < 5: return 'Bronze'

datamart['General_Segment'] = datamart.apply(segment_name, axis=1)
datamart.head(100)

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score,General_Segment
CustomerID,,,,,,,,,
12346.0,325,2,0.00,1,1,1,111,3,Bronze
12821.0,213,6,92.72,1,1,1,111,3,Bronze
13723.0,216,6,199.85,1,1,1,111,3,Bronze
12817.0,280,4,166.04,1,1,1,111,3,Bronze
15843.0,303,8,118.95,1,1,1,111,3,Bronze
...,...,...,...,...,...,...,...,...,...
13682.0,186,3,59.50,1,1,1,111,3,Bronze
15691.0,251,2,136.80,1,1,1,111,3,Bronze
15869.0,276,15,149.64,1,1,1,111,3,Bronze
